# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。

    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。

    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。

    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [12]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500)

In [13]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))




1. 光速科技（LightSpeed Technology Co.,Ltd）
2. 火箭显卡（Rocket Graphics Co.,Ltd）
3. 极限视界（Limitless Vision Co.,Ltd）
4. 天马行空（Celestial Computing Co.,Ltd）
5. 弹丸图形（Bullet Graphics Co.,Ltd）
6. 神威计算（Divine Computing Co.,Ltd）
7. 超越科技（Surpassing Technology Co.,Ltd）
8. 雷霆显卡（Thunder Graphics Co.,Ltd）
9. 精英加速（Elite Acceleration Co.,Ltd）
10. 无限创想（Infinite Creativity Co.,Ltd）


In [4]:
chain.verbose = True

In [5]:
chain.verbose

True

In [6]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.


1. 创新科技股份有限公司
2. 强势图形技术有限公司
3. 超视觉半导体有限公司
4. 光电视界有限公司
5. 典范图像处理有限公司
6. 逸杰计算机图形有限公司
7. 元良电脑图形有限公司
8. 集实数字图形有限公司
9. 久通计算机图形有限公司
10. 昱轩图像技术有限公司


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [7]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [8]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [9]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [10]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...

《三体人不是无法战胜的》是一部关于人类与外星种族之间的激烈对抗的戏剧。故事发生在未来的地球，随着科技的进步，人类与三体人这一外星种族之间的联系日渐增多。然而，三体人并非像人类所想象的那样无法战胜，他们拥有强大的科技实力和超乎想象的智慧。在与三体人的不断交锋中，人类面临着前所未有的挑战和危机。但是，主人公们并没有放弃，他们发挥出人类的智慧和勇气，与三体人展开了一场殊死搏斗。在这场生死之战中，人类是否能够战胜自己的惧怕和不断进化的敌人？最终，谁能够成为最终的胜利者？《三体人不是无法战胜的》将为观众带来一场惊险刺激的战斗，同时也探讨了人类的勇气和智慧面对未知挑战的力量。
 

《三体人不是无法战胜的》是一部充满惊险和刺激的戏剧作品，它将观众带入了一个科幻世界，探讨了人类与外星种族之间的激烈对抗。

故事发生在未来的地球，随着科技的进步，人类与三体人之间的联系日渐增多。然而，三体人并非像人类所想象的那样无法战胜，他们拥有强大的科技实力和超乎想象的智慧。这一点也体现在剧中，三体人不断利用他们的科技优势与人类展开激烈的对抗，让人类陷入了前所未有的挑战和危机之中。

但是，主人公们并没有因此而放弃，他们发挥出人类的智慧和勇气，与三体人展开了一场殊死搏斗。在这场生死之战中，人类不仅仅是在与一个外星种族的战斗，更是在与自己的惧怕和不断进化的敌人作战。剧中充满了紧张刺激的场面，让观众仿佛置身于战场之中，能够感受到主人公们的勇气和决心。

除了惊险刺激的战斗场面，该剧也探讨了人类面对未知挑战时的力量。主人公们不断面对困难和挑战，但是他们从未放弃，而是用智慧和勇气克服了一切。这也是人类一直以来的精神，无论面对怎样的困难，我们都会勇敢地迎接挑战并最终取得胜利。

总的来说，《三体人不是无法战胜的》是一部精彩的戏剧作品，它不仅带来了惊险刺激的战斗场面，更让我们思考人类的勇气和智慧面对未知挑战的力量。这部剧将给观众带来一场难忘的观剧体验，让我们一起为主人公们的勇气和决心欢呼！

> Finished chain.


In [11]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满惊险刺激的科幻戏剧，它延续了经典系列的传奇故事。在这一季中，我们将见证新一代的英雄们挑战邪恶势力的勇气和智慧。随着军阀和反抗军的激烈战斗，我们将看到新的联盟形成，旧的敌人和朋友之间的纠葛也将达到高潮。同时，神秘的力量将再次出现，为战争增添更多的变数。在这场宏大的冒险中，我们将见证爱、友情、背叛和牺牲的力量如何影响我们的英雄们，最终决定他们的命运。《星球大战第九季》将带领观众穿越宇宙，探索力量与黑暗的较量，以及人性的光与影。这将是一场前所未有的精彩旅程，不容错过。


《星球大战第九季》是一部令人惊叹的科幻戏剧，它不仅延续了经典系列的传奇故事，更展现了新一代英雄们的勇气和智慧。导演们成功地将宏大的宇宙背景与细腻的人性冲突融合在一起，为观众呈现了一场前所未有的精彩旅程。

在本季中，军阀和反抗军的激烈战斗令人目不转睛，不仅有惊险刺激的动作场面，更有英雄们面对自我和命运的挑战。同时，新的联盟的形成也为剧情增添了更多的戏剧性。旧的敌人和朋友之间的复杂关系也将达到高潮，为故事增添了更多的张力。

除了精彩的战斗场面，观众还将在《星球大战第九季》中看到爱、友情、背叛和牺牲这些人性的光与影。这些情感的表现令人动容，也让观众更加投入剧情，与英雄们一起经历他们的挑战和成长。

最令人期待的是，神秘的力量再次出现，为故事增添更多的变数。这将是一场力量与黑暗的较量，也将成为故事的关键点，令观众充满期待。

总的来说，《星球大战第九季》是一部不容错过的戏剧，它将带领观众穿越宇宙，探索力量与黑暗的较量，以及人性的光与影。导演们成功地将科幻元素与情感冲突融合在一起，为观众呈现了一场令人惊叹的冒险。不论您是《星球大战》的忠实粉丝，还是喜欢惊险刺激的戏剧，都不应错过这部精彩的作品。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [12]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [13]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [14]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [15]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：


/home/ubuntu/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


在二十一世纪的新中国，人类面临着前所未有的挑战。随着科技的进步，人类发现了一个神秘的外星文明——三体人。这个文明拥有超强的科技实力，他们的存在让人类感到恐慌和无力。

然而，一群勇敢的中国科学家和军人决定挑战命运，与三体人展开一场生死决战。在这场战斗中，他们不仅要面对三体人强大的科技，还要面对内心的挣扎和矛盾。他们必须找到战胜三体人的方法，否则人类将面临灭亡的威胁。

在这场战斗中，勇气、智慧、坚持不懈的精神将被激发出来，人类将展现出惊人的力量。最终，他们能否战胜三体人，保卫地球，成为二十一世纪新中国的英雄？这将是一场跨越时空的壮丽战争，也是人类文明的终极挑战。三体人不是无法战胜的，只要人类不放弃希望，就能战胜一切困难。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n在二十一世纪的新中国，人类面临着前所未有的挑战。随着科技的进步，人类发现了一个神秘的外星文明——三体人。这个文明拥有超强的科技实力，他们的存在让人类感到恐慌和无力。\n\n然而，一群勇敢的中国科学家和军人决定挑战命运，与三体人展开一场生死决战。在这场战斗中，他们不仅要面对三体人强大的科技，还要面对内心的挣扎和矛盾。他们必须找到战胜三体人的方法，否则人类将面临灭亡的威胁。\n\n在这场战斗中，勇气、智慧、坚持不懈的精神将被激发出来，人类将展现出惊人的力量。最终，他们能否战胜三体人，保卫地球，成为二十一世纪新中国的英雄？这将是一场跨越时空的壮丽战争，也是人类文明的终极挑战。三体人不是无法战胜的，只要人类不放弃希望，就能战胜一切困难。',
 'review': '\n\n《三体》是一部令人叹为观止的戏剧作品。它不仅仅是一场关于科技与文明的较量，更是一场关于人性的探讨。\n\n剧中的主人公们，一群普通的中国科学家和军人，面对着前所未有的危机和挑战，他们的内心矛盾和挣扎令人动容。他们不仅要面对外来的威胁，还要面对自己内心的恐惧和不安。但他们并没有放弃，而是勇敢地挑战命运，寻找战胜三体人的方法。\n\n这部戏剧不仅仅是一场惊心动魄的科幻冒险，更是一次心灵的洗礼。它让我们看到了人类在面对困难时的勇气和坚持不懈的精神。它也让我们思考人类文明的未来，以及我们与其他文明的交流和合作。\n\n剧中的舞台设计和音效都非常出色，让观众仿佛置身于未来的世界。演员们的精彩表演更是让整部剧目生动而感人。\n\n总的来说，《三体》不仅是一部娱乐性强的戏剧作品，更是一部富有深意的作品。它让观众在欣赏的同时，也能思考人类的命运和文明的发展。我相信它将会成为一部经久不衰的经典作品，值得观众们反复品味。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果